<a href="https://colab.research.google.com/github/NicholsonCameron/Lab-1.1/blob/main/comparing_league_wide_ERA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [64]:
import polars as pl

In [65]:
pitching = pl.read_csv('./Data/lahman_1871-2023_csv/Pitching.csv')
pitching.head()

playerID,yearID,stint,teamID,lgID,W,L,G,GS,CG,SHO,SV,IPouts,H,ER,HR,BB,SO,BAOpp,ERA,IBB,WP,HBP,BK,BFP,GF,R,SH,SF,GIDP
str,i64,i64,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
"""aardsda01""",2004,1,"""SFN""","""NL""",1,0,11,0,0,0,0,32,20,8,1,10,5,0.417,6.75,0,0,2,0,61,5,8,0,1,1
"""aardsda01""",2006,1,"""CHN""","""NL""",3,0,45,0,0,0,0,159,41,24,9,28,49,0.214,4.08,0,1,1,0,225,9,25,1,3,2
"""aardsda01""",2007,1,"""CHA""","""AL""",2,1,25,0,0,0,0,97,39,23,4,17,36,0.3,6.4,3,2,1,0,151,7,24,2,1,1
"""aardsda01""",2008,1,"""BOS""","""AL""",4,2,47,0,0,0,0,146,49,30,4,35,49,0.268,5.55,2,3,5,0,228,7,32,3,2,4
"""aardsda01""",2009,1,"""SEA""","""AL""",3,6,73,0,0,0,38,214,49,20,4,34,80,0.19,2.52,3,2,0,0,296,53,23,2,1,2


In [77]:
ERA = (pitching 
       .select([pl.col('IPouts'), pl.col('R'), pl.col('lgID'), pl.col('yearID')])
       .filter(pl.col('yearID') >= 1946)
       .group_by(pl.col('yearID'), pl.col('lgID'))
       .agg([pl.col('IPouts').sum().alias('total_outs'),
             pl.col('R').sum().alias('total_runs'),
            ])
       .with_columns(era = (pl.col('total_runs')/pl.col('total_outs')*27))
        )
ERA.head()

yearID,lgID,total_outs,total_runs,era
i64,str,i64,i64,f64
1953,"""NL""",32915,5914,4.851223
1977,"""AL""",60672,10247,4.560077
2007,"""NL""",69742,12388,4.795905
1960,"""NL""",33361,5250,4.248973
1955,"""NL""",32953,5578,4.570327


In [41]:
ERA.write_csv('./data/ERA.csv')

In [67]:
batting = pl.read_csv('./Data/lahman_1871-2023_csv/Batting.csv')
batting.head()

playerID,yearID,stint,teamID,lgID,G,G_batting,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP,G_old
str,i64,i64,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str
"""aardsda01""",2004,1,"""SFN""","""NL""",11,null,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,null
"""aardsda01""",2006,1,"""CHN""","""NL""",45,null,2,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,null
"""aardsda01""",2007,1,"""CHA""","""AL""",25,null,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,null
"""aardsda01""",2008,1,"""BOS""","""AL""",47,null,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,null
"""aardsda01""",2009,1,"""SEA""","""AL""",73,null,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,null


In [79]:
SLG = (batting
       .select([pl.col('BB'), pl.col('2B'), pl.col('3B'), pl.col('HR'), 
                pl.col('AB'), pl.col('yearID'), pl.col('lgID')])
       .filter(pl.col('yearID') >= 1946)
       .group_by(pl.col('yearID'), pl.col('lgID'))
       .agg([pl.col('BB').sum().alias('total_BB'),
             pl.col('2B').sum().alias('total_2B'),
             pl.col('3B').sum().alias('total_3B'),
             pl.col('HR').sum().alias('total_HR'),
             pl.col('AB').sum().alias('total_AB'),
            ])
       .with_columns(slg = (pl.col('total_BB') + (2 * pl.col('total_2B')) + 
                            (3 * pl.col('total_3B')) + (4 * pl.col('total_HR')))/pl.col('total_AB'))
      )
SLG.head()

yearID,lgID,total_BB,total_2B,total_3B,total_HR,total_AB,slg
i64,str,i64,i64,i64,i64,i64,f64
1960,"""NL""",3937,1722,384,1042,42176,0.301143
1992,"""NL""",5978,2967,459,1262,65748,0.278898
2007,"""NL""",8576,4898,505,2705,89489,0.343137
2010,"""AL""",7367,4016,368,2209,77467,0.327094
1998,"""AL""",7737,4248,408,2499,78416,0.350094
